In [1]:
import numpy as np
np.random.seed(13)

import keras.backend as K
from keras.models import Sequential
from keras.layers import Dense, Embedding, Lambda
from keras.utils.data_utils import get_file
from keras.utils import np_utils
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer

import gensim

Using TensorFlow backend.


In [2]:
path = get_file('alice.txt', origin='http://www.gutenberg.org/files/11/11-0.txt')
corpus = open(path).readlines()[:300]
corpus = [sentence for sentence in corpus if sentence.count(' ') >= 2]

tokenizer = Tokenizer()
tokenizer.fit_on_texts(corpus)
corpus = tokenizer.texts_to_sequences(corpus)
nb_samples = sum(len(s) for s in corpus)
V = len(tokenizer.word_index) + 1
dim = 100
window_size = 2

In [3]:
def generate_data(corpus, window_size, V):
    maxlen = window_size*2
    for words in corpus:
        L = len(words)
        for index, word in enumerate(words):
            contexts = []
            labels   = []            
            s = index - window_size
            e = index + window_size + 1
            
            contexts.append([words[i] for i in range(s, e) if 0 <= i < L and i != index])
            labels.append(word)

            x = sequence.pad_sequences(contexts, maxlen=maxlen)
            y = np_utils.to_categorical(labels, V)
            yield (x, y)

In [4]:
cbow = Sequential()
cbow.add(Embedding(input_dim=V, output_dim=dim, input_length=window_size*2))
cbow.add(Lambda(lambda x: K.mean(x, axis=1), output_shape=(dim,)))
cbow.add(Dense(V, activation='softmax'))

In [5]:
cbow.compile(loss='categorical_crossentropy', optimizer='adadelta')

In [6]:
for ite in range(10):
    loss = 0.
    for x, y in generate_data(corpus, window_size, V):
        loss += cbow.train_on_batch(x, y)

    print(ite, loss)

0 17897.1324615
1 16611.4819975
2 16054.5737875
3 15975.4586413
4 16078.7989137
5 16168.2691295
6 16193.6849765
7 16191.0186955
8 16183.2132714
9 16174.5272493


In [7]:
f = open('vectors.txt' ,'w')
f.write('{} {}\n'.format(V-1, dim))

8

In [8]:
vectors = cbow.get_weights()[0]
for word, i in tokenizer.word_index.items():
    str_vec = ' '.join(map(str, list(vectors[i, :])))
    f.write('{} {}\n'.format(word, str_vec))
f.close()

In [9]:
w2v = gensim.models.KeyedVectors.load_word2vec_format('./vectors.txt', binary=False)

In [10]:
w2v.most_similar(positive=['the'])

[('a', 0.8029883503913879),
 ('her', 0.6420163512229919),
 ('i', 0.6390146613121033),
 ('it', 0.6209530830383301),
 ('this', 0.6207232475280762),
 ('my', 0.6043308973312378),
 ('you', 0.5718163847923279),
 ('to', 0.5632857084274292),
 ('about', 0.5545670986175537),
 ('down', 0.5478866696357727)]

In [11]:
w2v.most_similar(positive=['alice'])

[('said', 0.522140383720398),
 ('she', 0.4846324920654297),
 ('thought', 0.4466657042503357),
 ('you', 0.4399128556251526),
 ('say', 0.4043766260147095),
 ('now', 0.40042510628700256),
 ('her', 0.3967324495315552),
 ('that', 0.38552236557006836),
 ('because', 0.37965667247772217),
 ('dark', 0.3769078254699707)]